In [ ]:
# Imports

In [ ]:
# Filter columns to the most important variables (Feature Importance)

In [ ]:
# Compare different binary classifier models:
# - tree based
# - logistic regression
# - knn 
# - naive bayes

In [ ]:
# Show feature importance